# Interactive Opioid Crisis Visualizer

### Settings

In the cell below, you can choose which drug(s) you want to display data for. 

You can also choose to enable or disable video exporting of the visualization.

In [ ]:
# Select at least one drug to visualize
benzos               = False
cocaine              = True
fentanyl             = True
heroin               = True
natsemi              = False
prescription_opioids = False
psychostimulants     = False

# Select any start and end year to visualize, between 2011 and 2022
start_year           = 2011
end_year             = 2022
    
# Options for exporting visualizations
title                = 'OpioidDeaths' # Replace with your own
export_video         = True           # Enable if you want a .mp4 video of the visualization
dpi                  = 600
export_images        = True           # Enable if you want each of the images exported as a .jpg file.

### Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, filters, morphology
from skimage.color import label2rgb
from skimage.color import rgb2gray, gray2rgb
from skimage import measure
from utils/ import *
from matplotlib.animation import FFMpegWriter
# plt.rcParams['animation.ffmpeg_path'] ='C:\\ffmpeg\\bin\\ffmpeg.exe' # I need this for FFMPEG to run on my machine
%matplotlib qt
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Some checks to make sure data works

assert start_year <= end_year, "Please select a valid range."
assert start_year >= 2011, "Please select a starting year between 2011 and 2022."
assert end_year   <= 2022, "Please select an ending year between 2011 and 2022."

options = [benzos, cocaine, fentanyl, heroin, natsemi, prescription_opioids, psychostimulants]
choices = [1 if choice else 0 for choice in options]
assert any(options), "Please select at least one drug to visualize."

In [ ]:
# Strings

all_names = ["Benzodiazepines", "Cocaine", "Fentanyl", "Heroin", "Natural or Semi-Synthetic Opioids", "Prescription Opioids", "Psychostimulants"]
names = []
for i in range(len(options)):
    if options[i]:
        names.append(all_names[i])

n_choices = len(names)

selected_names = ""

if n_choices > 1:
    for i in range(n_choices-1):
        selected_names += names[i] + ", "

selected_names += names[n_choices-1]

title_mp4 = title + ".mp4"

In [ ]:
# Read in data from csv files
benzos_df = pd.read_csv('./data/benzos.csv')
cocaine_df = pd.read_csv('./data/cocaine.csv')
fentanyl_df = pd.read_csv('./data/fentanyl.csv')
heroin_df = pd.read_csv('./data/heroin.csv')
natsemi_df = pd.read_csv('./data/natsemi.csv')
prescription_df = pd.read_csv('./data/prescription.csv')
psychostimulant_df = pd.read_csv('./data/psychostimulant.csv')

all_dataframes = [benzos_df, cocaine_df, fentanyl_df, heroin_df, natsemi_df, prescription_df, psychostimulant_df]
dataframes = []
for i in range(len(options)):
    if options[i]:
        dataframes.append(all_dataframes[i])

### Heat Map Visualization

In this section, we turn an image of Alameda County, divided by zip codes, into a heat map displaying the death rates of the chosen drugs.

In [ ]:
alameda = io.imread('./data/better_alameda.jpg')
gray_alameda = rgb2gray(alameda)
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
plt.axis=('off')
plt.imshow(alameda, cmap='gray')
plt.show()

print("Original image, for reference.")

In [ ]:
# Define the percentage thresholds
percent1 = 30
val1 = gray_alameda.max() * percent1 * 0.01
percent2 = 60
val2 = gray_alameda.max() * percent2 * 0.01

# Create a binary mask based on the thresholds
alameda_bw = (gray_alameda > val1) & (gray_alameda < val2)

# Perform binary erosion to remove small specks
alameda_bw_eroded = morphology.binary_erosion(alameda_bw, morphology.disk(2.5))


# Plot the results
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.set_axis_off()
plt.imshow(alameda_bw_eroded, cmap='gray')
plt.show()

print("Enhanced black and white image with smoothed edges")


In [ ]:

labels = measure.label(alameda_bw_eroded)

props = measure.regionprops(labels)

print(f"Originally detecting {len(np.unique(labels))} zip codes.")



In [ ]:
overlay = label2rgb(labels, image=alameda_bw_eroded)

fig, ax = plt.subplots(1, 1, figsize=(15, 15))
plt.axis=('off')
plt.imshow(overlay)
plt.show()

In [ ]:
# This is just a mapping of each component of the image to its corresponding zip code
# Feel free to ignore
areas_map = [
    94708,
    94707,
    94706,
    94706,
    94710,
    94709,
    94703,
    94720,
    94702,
    94702,
    94704,
    94705,
    94705,
    94618,
    94611,
    94609,
    94511,
    94608,
    94607,
    94588,
    94610,
    94602,
    94612,
    94546,
    94546,
    94546,
    94606,
    94619,
    94552,
    94501,
    94601,
    94613,
    94605,
    94621,
    94621,
    94603,
    94502,
    94550,
    94557,
    94568,
    94578,
    94579,
    94580,
    94550,
    94541,
    94566,
    94542,
    94544,
    94545,
    94586,
    94587,
    94536,
    94539,
    94555,
    94538,
    94560,
    94560,
    94560,
]

# Unfortunately, this had to be done manually

#### Heat map

In [ ]:
# Task: display overdose data on the map as if it were a heat map
img = np.zeros(alameda_bw_eroded.shape, dtype=float)
years = range(start_year, end_year + 1) # Display all years from
colormap = 'hot'

fig = plt.figure()

if export_video:
    # Video setup
    metadata = dict(title='Overdoses', artist='Alex Sanchez', comment='Xylazine is coming.')
    writer = FFMpegWriter(fps=15, metadata=metadata)
    with writer.saving(fig, f"./out/{title_mp4}", dpi=dpi):
        for year in years:
            for i in range(len(props)):
                area = props[i]
                zip_code = areas_map[i]
                death_rate = 1
                for df in dataframes:
                    df_death_rate = df.loc[(df['Year'] == year) & (df['Zip Code'] == zip_code), 'Age-Adjusted Rate']
                    # Check if any values are present
                    if not df_death_rate.empty:
                        df_death_rate = df_death_rate.values[0]
                    else:
                        df_death_rate = 0.0
                    death_rate += df_death_rate
                
                x1, y1, x2, y2 = area.bbox
                img[x1:x2, y1:y2] += area.image * (death_rate)

            
            plt.clf()
            plt.title(f'Overdose Heatmap - Year {year} - {selected_names}')
            plt.imshow(img, cmap=colormap)
            cbar = plt.colorbar()
            cbar.set_label('Death Rate')  # Set your desired label here
            plt.show()
            plt.draw()
            plt.pause(0.5)
            writer.grab_frame()
            if (export_images):        
                plt.savefig(f"./out/{title}_{year}.jpg", format='jpg', transparent=True, dpi=dpi)

else:
    for year in years:
        for i in range(len(props)):
            area = props[i]
            zip_code = areas_map[i]
            death_rate = 1
            for df in dataframes:
                df_death_rate = df.loc[(df['Year'] == year) & (df['Zip Code'] == zip_code), 'Age-Adjusted Rate']
                # Check if any values are present
                if not df_death_rate.empty:
                    df_death_rate = df_death_rate.values[0]
                else:
                    df_death_rate = 0.0
                death_rate += df_death_rate
            
            x1, y1, x2, y2 = area.bbox
            img[x1:x2, y1:y2] += area.image * (death_rate)

        
        plt.clf()
        plt.title(f'Overdose Heatmap - Year {year} - {selected_names}')
        plt.imshow(img, cmap=colormap)
        cbar = plt.colorbar()
        cbar.set_label('Death Rate')  # Set your desired label here
        plt.show()
        if (export_images):        
            plt.savefig(f"./out/{title}_{year}.jpg", format='jpg', transparent=True, dpi=dpi)


